In [59]:
import numpy as np
import random
from mnist import MNIST
import matplotlib.pyplot as plt

In [58]:
mndata = MNIST('../HW1/Dataset')
trainX = np.array(mndata.load_training()[0])[:50000]
trainY = np.array(mndata.load_training()[1])[:50000]

testX = np.array(mndata.load_testing()[0])[:1000]
testY = np.array(mndata.load_testing()[1])[:1000]

In [60]:
valIndices = np.random.choice(len(trainX), 2000)
nonValIndices = [x for x in range(len(trainX)) if x not in valIndices]

valX = trainX[valIndices]
valY = trainY[valIndices]

trainX = trainX[nonValIndices]
trainY = trainY[nonValIndices]

testX = np.array(mndata.load_testing()[0])[:2000]
testY = np.array(mndata.load_testing()[1])[:2000]

def feat(data,i):
    return data[i].tolist()

def oneHot(clas, noOfClasses):
    feat = np.zeros(noOfClasses)
    feat[clas] = 1;
    return feat

trnX = np.array([feat(trainX,i) for i in range(trainX.shape[0])])/256.0
trnY = np.array([oneHot(trainY[i], 10) for i in range(trainX.shape[0])])

tstX = np.array([feat(testX,i) for i in range(testX.shape[0])])/256.0
tstY = np.array([oneHot(testY[i], 10) for i in range(testX.shape[0])])

valX = np.array([feat(valX,i) for i in range(valX.shape[0])])/256.0
valY = np.array([oneHot(valY[i], 10) for i in range(valX.shape[0])])

In [63]:
def sigmoid(x):
    return 1.0/(1+np.exp(-1*x))

def lecun(x):
    return 1.7159*np.tanh(2.0*x/3)

def grad(x):
    return x*(1-x)

def gradLecun(x):
    t = 2.0*x/3
    return 1.7159*(1-t**2)

def softmax(x):
    x = np.exp(x)
    x = x/x.sum(axis=1)[:, None]
    return x
    
maxValAcc = 0
lr = 0.00001
trnAcc = []
valAcc = []
tstAcc = []

# randomly initialize our weights with mean 0
n_hid_1 = 100
fan_in = 1.0#/np.sqrt(784)
# print fan_in
W1 = fan_in*(2*np.random.random((784,n_hid_1)) - 1)
bias = fan_in*(2*np.random.random((n_hid_1)) - 1)
fan_in_h = 1.0#/np.sqrt(n_hid_1)
W2 = fan_in_h*(2*np.random.random((n_hid_1,10)) - 1)
bias2 = fan_in_h*(2*np.random.random((10)) - 1)

for j in xrange(300):
    indices = np.random.choice(len(trnX), len(trnX))
    tempY = trnY[indices]
    tempX = trnX[indices]
    A1 = np.dot(tempX, W1) + bias
    l1 = sigmoid(A1)
    
    A2 = np.dot(l1, W2) + bias2
    l2 = softmax(A2)

    # Errors in output layer
    d2 = (l2 - tempY)
    dbias2 = np.sum(d2, axis = 0)
        
    # Delta of W2
    dW2 = np.dot(l1.T, d2)

    # Errors in 1st hidden layer
    d1 = np.dot(d2, W2.T)*grad(l1)
    dbias = np.sum(d1, axis = 0)    
    
    # Delta W2
    dW1 = np.dot(tempX.T, d1)
    
    W2 -= lr*dW2
    bias2 -= lr*dbias2
    W1 -= lr*dW1
    bias -= lr*dbias
    
    prediction = softmax(np.dot(sigmoid(np.dot(valX, W1)+bias), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(valY, axis = 1), np.argmax(prediction, axis = 1))]
    valAcc.append(np.sum(correct)*100.0/len(valX))
    
    prediction = softmax(np.dot(sigmoid(np.dot(trnX, W1)+bias), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(trnY, axis = 1), np.argmax(prediction, axis = 1))]
    trnAcc.append(np.sum(correct)*100.0/len(trnX))   
    
    prediction = softmax(np.dot(sigmoid(np.dot(tstX, W1)+bias), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(tstY, axis = 1), np.argmax(prediction, axis = 1))]
    tstAcc.append(np.sum(correct)*100.0/len(tstX))   
    
    print tstAcc[-1]

8.9
16.15
17.85
23.75
27.25
29.3
32.55
34.8
36.95
38.8
40.25
41.9
43.8
45.4
46.6
48.15
49.3
50.95
51.75
52.55
53.15
54.35
55.3
55.85
56.4
57.35
58.35
58.9
59.35
60.4
60.7
61.65
62.05
62.55
63.4
63.75
64.35
64.45
64.85
65.15
65.7
66.05
66.4
66.5
66.9
67.05
67.15
67.65
67.65
68.15
68.3
68.9
69.2
69.3
69.4
69.5
69.8
70.05
70.15
70.2
70.3
70.45
70.6
70.95
71.15
71.45
71.65
71.95
72.05
72.3
72.3
72.45
72.7
72.95
73.25
73.3
73.55
73.6
73.65
73.85
74.0
74.35
74.25
74.65
74.7
74.8
74.85
74.95
75.2
75.1
75.5
75.4
75.6
75.65
75.85
75.95
76.25
76.4
76.3
76.25
76.5
76.8
76.75
76.85
77.1
77.2
77.5
77.55
77.7
77.95
77.8
78.15
78.25
78.4
78.5
78.45
78.5
78.7
78.55
78.65
78.7
78.8
78.9
78.85
79.15
79.1
79.25
79.15
79.25
79.15
79.4
79.55
79.75
79.55
79.8
79.9
79.8
79.95
80.0
80.05
79.9
80.15
80.15
80.2
80.15
80.25
80.25
80.3
80.5
80.55
80.4
80.45
80.75
80.65
80.7
80.85
80.9
81.1
81.1
80.95
81.05
81.1
81.2
81.25
81.3
81.3
81.35
81.35
81.4
81.5
81.55
81.5
81.55
81.55
81.7
81.85
81.9
81.85
81.9
82.1
82.1


In [64]:
print "Final test accuracy = ", tstAcc[-1]
print "Final training accuracy =", trnAcc[-1]
print "Final validation accuracy = ", valAcc[-1]
plt.plot([x+1 for x in range(len(trnAcc))], trnAcc, label = 'Training Accuracy')
plt.plot([x+1 for x in range(len(valAcc))], valAcc, label = 'Validation Accuracy')
plt.plot([x+1 for x in range(len(tstAcc))], tstAcc, label = 'Testing Accuracy')
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.legend(loc='lower right', shadow=True)
plt.show()

Final test accuracy =  84.95
Final training accuracy = 86.6544523329
Final validation accuracy =  86.35


In [70]:
def error(trnY, y):
    err = 0    
    for i in range(len(trnY)):
        for k in range(10):
            if(y[i, k] < 0.00001):
                y[i, k] = 0.00001
            elif(y[i, k] > 0.99999):
                y[i, k] = 0.99999
            err += trnY[i, k]*np.log(y[i, k])
    err=-1*err/len(trnY)    
    return err

In [86]:
indices = np.random.choice(len(trnX), 3)
d = trnX[indices]
t = trnY[indices]
epsilon = 0.01
for (i, j) in np.ndindex(W2.shape):
    e = np.zeros(W2.shape)
    e[i, j] = 0.01
    A1 = np.dot(d, W1) + bias
    l1 = sigmoid(A1)
    A2 = np.dot(l1, W2-e) + bias2
    l2 = softmax(A2)
    # Errors in output layer
    d2 = (l2 - t)
    dbias2 = np.sum(d2, axis = 0)
    # Delta of W2
    dW2 = np.dot(l1.T, d2)
    # Errors in 1st hidden layer
    d1 = np.dot(d2, W2.T-e.T)*grad(l1)
    dbias = np.sum(d1, axis = 0)    
    # Delta W2
    dW1 = np.dot(d.T, d1)

    ErrMinus = error(t, l2)

    A1 = np.dot(d, W1) + bias
    l1 = sigmoid(A1)
    A2 = np.dot(l1, W2+e) + bias2
    l2 = softmax(A2)
    # Errors in output layer
    d2 = (l2 - t)
    dbias2 = np.sum(d2, axis = 0)
    # Delta of W2
    dW2 = np.dot(l1.T, d2)
    # Errors in 1st hidden layer
    d1 = np.dot(d2, W2.T+e.T)*grad(l1)
    dbias = np.sum(d1, axis = 0)    
    # Delta W2
    dW1 = np.dot(d.T, d1)

    ErrPlus = error(t, l2)

    A1 = np.dot(d, W1) + bias
    l1 = sigmoid(A1)
    A2 = np.dot(l1, W2) + bias2
    l2 = softmax(A2)
    # Errors in output layer
    d2 = (l2 - t)
    dbias2 = np.sum(d2, axis = 0)
    # Delta of W2
    dW2 = np.dot(l1.T, d2)
    # Errors in 1st hidden layer
    d1 = np.dot(d2, W2.T)*grad(l1)
    dbias = np.sum(d1, axis = 0)    
    # Delta W2
    dW1 = np.dot(d.T, d1)
    
    print dW2[i][j] - (ErrPlus-ErrMinus)/2*epsilon


4.03082098922e-06
1.18690719832e-05
0.00606566358064
0.000843308162776
0.0915087250892
0.00654418302567
-0.106646802689
0.000469730848065
0.000515425397626
0.000683866687824
-0.00320801231662
1.85332385752e-05
8.01197438509e-05
2.49034610306e-06
4.74761669009e-05
0.00296772113779
-1.69635663304e-05
6.0978333189e-05
1.73418662913e-06
4.59227299127e-05
-4.41809924746e-05
1.19036487008e-05
0.00595173537683
0.000827378311213
0.0897735424258
0.00646467342418
-0.104623990724
0.000461691026361
0.000505703936856
0.000671543562521
-0.0252096126559
9.57903390087e-05
0.00677463396469
0.00100749601051
0.0964810694609
0.0305443313404
-0.112118591434
0.000822767328639
0.000656824109573
0.000945291531777
-0.00144523125341
5.87714670596e-07
7.05826150536e-05
2.28147690418e-05
0.000972170697342
0.00144877589968
-0.00111298120438
1.33996430318e-05
1.90209965628e-05
1.08601224042e-05
-0.015953201103
5.17741728704e-05
0.00024248825623
7.72192707852e-05
8.75918234939e-05
0.014971690733
0.000102681496434
0.

In [80]:
print ErrPlus

1.39021681152


In [81]:
print ErrMinus

2.05175014143
